In [194]:
import nltk
import numpy as np
import random
import string
# import matplotlib.pyplot as plt

import bs4 as bs
import urllib.request
import re
from tkinter import *

In [80]:
raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Cricket')
raw_html = raw_html.read()

article_html = bs.BeautifulSoup(raw_html, 'html')

article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [81]:
article_text

'\ncricket is a bat-and-ball game played between two teams of eleven players on a field at the centre of which is a 22-yard (20-metre) pitch with a wicket at each end, each comprising two bails balanced on three stumps. the batting side scores runs by striking the ball bowled at the wicket with the bat (and running between the wickets), while the bowling and fielding side tries to prevent this (by getting the ball to either wicket) and dismiss each batter (so they are "out"). means of dismissal include being bowled, when the ball hits the stumps and dislodges the bails, and by the fielding side catching the ball after it is hit by the bat, but before it hits the ground. when ten batters have been dismissed, the innings ends and the teams swap roles. the game is adjudicated by two umpires, aided by a third umpire and match referee in international matches. they communicate with two off-field scorers who record the match\'s statistical information.\nforms of cricket range from twenty20, 

In [82]:
article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
article_text = re.sub(r'\s+', ' ', article_text)


In [83]:
article_sentences = nltk.sent_tokenize(article_text)
article_words = nltk.word_tokenize(article_text)

In [84]:
article_sentences

[' cricket is a bat-and-ball game played between two teams of eleven players on a field at the centre of which is a 22-yard (20-metre) pitch with a wicket at each end, each comprising two bails balanced on three stumps.',
 'the batting side scores runs by striking the ball bowled at the wicket with the bat (and running between the wickets), while the bowling and fielding side tries to prevent this (by getting the ball to either wicket) and dismiss each batter (so they are "out").',
 'means of dismissal include being bowled, when the ball hits the stumps and dislodges the bails, and by the fielding side catching the ball after it is hit by the bat, but before it hits the ground.',
 'when ten batters have been dismissed, the innings ends and the teams swap roles.',
 'the game is adjudicated by two umpires, aided by a third umpire and match referee in international matches.',
 "they communicate with two off-field scorers who record the match's statistical information.",
 'forms of cricket

In [85]:
article_words

['cricket',
 'is',
 'a',
 'bat-and-ball',
 'game',
 'played',
 'between',
 'two',
 'teams',
 'of',
 'eleven',
 'players',
 'on',
 'a',
 'field',
 'at',
 'the',
 'centre',
 'of',
 'which',
 'is',
 'a',
 '22-yard',
 '(',
 '20-metre',
 ')',
 'pitch',
 'with',
 'a',
 'wicket',
 'at',
 'each',
 'end',
 ',',
 'each',
 'comprising',
 'two',
 'bails',
 'balanced',
 'on',
 'three',
 'stumps',
 '.',
 'the',
 'batting',
 'side',
 'scores',
 'runs',
 'by',
 'striking',
 'the',
 'ball',
 'bowled',
 'at',
 'the',
 'wicket',
 'with',
 'the',
 'bat',
 '(',
 'and',
 'running',
 'between',
 'the',
 'wickets',
 ')',
 ',',
 'while',
 'the',
 'bowling',
 'and',
 'fielding',
 'side',
 'tries',
 'to',
 'prevent',
 'this',
 '(',
 'by',
 'getting',
 'the',
 'ball',
 'to',
 'either',
 'wicket',
 ')',
 'and',
 'dismiss',
 'each',
 'batter',
 '(',
 'so',
 'they',
 'are',
 '``',
 'out',
 "''",
 ')',
 '.',
 'means',
 'of',
 'dismissal',
 'include',
 'being',
 'bowled',
 ',',
 'when',
 'the',
 'ball',
 'hits',
 'the

In [86]:
wnlemmatizer = nltk.stem.WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [wnlemmatizer.lemmatize(token) for token in tokens]

punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

In [87]:

greeting_inputs = ("hey", "good morning", "good evening", "morning", "evening", "hi", "whatsup")
greeting_responses = ["hey", "hey hows you?", "*nods*", "hello, how you doing", "hello", "Welcome, I am good and you"]

def generate_greeting_response(greeting):
    for token in greeting.split():
        if token.lower() in greeting_inputs:
            return random.choice(greeting_responses)
        

In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [89]:
def generate_response(user_input):
    cricketrobo_response = ''
    article_sentences.append(user_input)

    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')
    all_word_vectors = word_vectorizer.fit_transform(article_sentences)
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
    similar_sentence_number = similar_vector_values.argsort()[0][-2]

    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        cricketrobo_response = cricketrobo_response + "I am sorry, I could not understand you"
        return cricketrobo_response
    else:
        cricketrobo_response = cricketrobo_response + article_sentences[similar_sentence_number]
        return cricketrobo_response

In [90]:
word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')
all_word_vectors = word_vectorizer.fit_transform(article_sentences)

In [91]:
similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

In [92]:
similar_sentence_number = similar_vector_values.argsort()[0][-2]

In [93]:
continue_dialogue = True
print("Hello, I am your friend Chatbot. You can ask me any question regarding Cricket:")
while(continue_dialogue == True):
    human_text = input()
    human_text = human_text.lower()
    if human_text != 'bye':
        if human_text == 'thanks' or human_text == 'thank you very much' or human_text == 'thank you':
            continue_dialogue = False
            print("Chatbot: Most welcome")
        else:
            if generate_greeting_response(human_text) != None:
                print("Chatbot: " + generate_greeting_response(human_text))
            else:
                print("Chatbot: ", end="")
                print(generate_response(human_text))
                article_sentences.remove(human_text)
    else:
        continue_dialogue = False
        print("Chatbot: Good bye and See you again...")

Hello, I am your friend Chatbot. You can ask me any question regarding Cricket:

Chatbot: I am sorry, I could not understand you
good morning
Chatbot: *nods*
morning
Chatbot: Welcome, I am good and you
soory
Chatbot: I am sorry, I could not understand you
bye
Chatbot: Good bye and See you again...


In [189]:
from tkinter import *  
from PIL import ImageTk,Image  
root = Tk()
root.title("cricket chatbot")
root.geometry("500x650")



def ask_from_bot():
    continue_dialogue = True
    return "Hello, I am your friend Chatbot. You can ask me any question regarding Cricket:"
    while(continue_dialogue == True):
        human_text = textF.get()
        human_text_lower = human_text.lower()
        if human_text_lower != 'bye':
            if human_text_lower == 'thanks' or human_text_lower == 'thank you very much' or human_text_lower == 'thank you':
                continue_dialogue = False
                answer_from_bot="Chatbot: Most welcome"
                msgs.insert(END, "you : " + human_text)
                print(type(answer_from_bot))
                msgs.insert(END, "bot : " + str(answer_from_bot))
                speak(answer_from_bot)
                textF.delete(0, END)
                msgs.yview(END)
            else:
                if generate_greeting_response(human_text_lower) != None: 
                    answer_from_bot="Chatbot: " + generate_greeting_response(human_text_lower)
                    msgs.insert(END, "you : " + human_text)
                    print(type(answer_from_bot))
                    msgs.insert(END, "bot : " + str(answer_from_bot))
                    speak(answer_from_bot)
                    textF.delete(0, END)
                    msgs.yview(END)
                else:
                    answer_from_bot=generate_response(human_text_lower)
                    article_sentences.remove(human_text_lower)
                    msgs.insert(END, "you : " + human_text)
                    print(type(answer_from_bot))
                    msgs.insert(END, "bot : " + str(answer_from_bot))
                    speak(answer_from_bot)
                    textF.delete(0, END)
                    msgs.yview(END)
        else:
            continue_dialogue = False
            answer_from_bot= "Chatbot: Good bye and See you again..."
            msgs.insert(END, "you : " + human_text)
            print(type(answer_from_bot))
            msgs.insert(END, "bot : " + str(answer_from_bot))
            speak(answer_from_bot)
            textF.delete(0, END)
            msgs.yview(END)
#     query = textF.get()
#     answer_from_bot = bot.get_response(query)
#     msgs.insert(END, "you : " + query)
#     print(type(answer_from_bot))
#     msgs.insert(END, "bot : " + str(answer_from_bot))
#     speak(answer_from_bot)
#     textF.delete(0, END)
#     msgs.yview(END)

load = Image.open("chatbot.jpg")
render = ImageTk.PhotoImage(load)
     
img = Label( image=render)
img.image = render
img.place(x=150, y=0)

img.pack(pady=5)

frame=Frame(root)

sc=Scrollbar(frame)

msgs=Listbox(frame, width=80,height=20)

sc.pack(side=RIGHT, fill=Y)

msgs.pack(side=LEFT, fill=BOTH, pady=10)

frame.pack()
# creating text field

textF = Entry(root, font=("Verdana", 10))
textF.pack(fill=X, pady=10)

btn = Button(root, text="Ask from bot", font=("Verdana", 20), command=ask_from_bot)
btn.pack()

root.mainloop()